<a href="https://colab.research.google.com/github/Jihyun-Eun/Bioinformatics/blob/main/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2. 웹크롤링 하기 전 사전지식

##2) 웹서버와 클라이언트

In [ ]:
import requests

response = requests.get("https://www.naver.com")
response.status_code #응답코드 확인

200

In [ ]:
#requtest 이용한 naver HTML 코드 출력

import requests

url = "https://www.naver.com"
response = requests.get(url) #requtest패키지에서 사용할 수 있는 .get메소드를 활용해
#url을 불러와 response 변수에 대입
#.get() http GET 요청을 보내는 메소드
# print(response) #<Response [200]>

#response에 담긴 내용을 text 메소드를 활용해 텍스트 형태로 변환후 html_text 변수에 대입
html_text = response.text
print(html_text)

   <!doctype html> <html lang="ko" class="fzoom"> <head> <meta charset="utf-8"> <meta name="Referrer" content="origin"> <meta http-equiv="X-UA-Compatible" content="IE=edge"> <meta name="viewport" content="width=1190"> <title>NAVER</title> <meta name="apple-mobile-web-app-title" content="NAVER"/> <meta name="robots" content="index,nofollow"/> <meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/> <meta property="og:title" content="네이버"> <meta property="og:url" content="https://www.naver.com/"> <meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png"> <meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/> <meta name="twitter:card" content="summary"> <meta name="twitter:title" content=""> <meta name="twitter:url" content="https://www.naver.com/"> <meta name="twitter:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png"> <meta name="twitter:description" c

#3. 웹크롤링 메소드

- 네이버 스포츠 야구 순위, 구단명 추출하기

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://sports.news.naver.com/kbaseball/record/index?category=kbo"
respose = requests.get(url)
# print(respose.status_code) 상태코드 확인

soup = bs(respose.text, 'html.parser')
# print(soup) #html 코드 확인

#find 메소드는 하나만 추출되기 때문에 .text 메소드를 바로 사용 가능함
# data = soup.find('h2', class_ = 'h_sch').text
# print(data)

#find_all 메소드는 리스트 형태로 추출되기 때문에 .text 메소드를 바로 사용불가함
#인덱스번호를 이용해 해당하는 인덱스를 지정해야함
data1 = soup.find_all('h2')
print(data1)

data2 = soup.find_all('h2')[0].text
print(data2)
data3 = soup.find_all('h2')[1].text
print(data3)

[<h2 class="blind">메인 메뉴</h2>, <h2 class="h_sch"><span>KBO리그 기록 및 순위</span></h2>]
메인 메뉴
KBO리그 기록 및 순위


In [ ]:
import csv

ranking_table = soup.find('tbody', id = "regularTeamRecordList_table")
#tbody 태그 안에 id 속성값이 한개만 있는 것 확인 후 ranking_table 변수에 대입
# print(ranking_table)

row = ranking_table.find_all('tr')
#ranking_table에서 가져온 것중에 tr태그로 되어있는 모든 것을 가져와 row에 대입
# print(row)

# for i in row:
# #row에 들어온 내용들을 하나씩 i에 대입(row는 리스트로 되어있음)
#     # print(i)
#     team = i.find_all('th')[0].text #i에 들어온 내용중 th태그의 첫번째를 내용만 team에 대입
#     name = i.find_all('div')[0].text #i에 들어온 내용중 div태그의 첫번째를 내용만 name에 대입
#     print(f"{team.strip()}위 : {name.strip()}")
#     #strip() 문자열 메소드, 양끝의 불필요한 문자열 제거하는 메소드

#csv 파일로 저장하기 위한 리스트 생성
data_list = []
for i in row:
    team = i.find_all('th')[0].text.strip()
    name = i.find_all('div')[0].text.strip()
    data_list.append([team, name])

#csv 파일로 저장
csv_file = "kbo_ranking.csv"
with open(csv_file, mode ="w", newline ='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['순위', '팀명'])
    writer.writerows(data_list)

print(f"데이터가 {csv_file}파일로 성공적으로 저장되었습니다")


데이터가 kbo_ranking.csv파일로 성공적으로 저장되었습니다


## 멜론 차트 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://www.melon.com/chart/index.htm"

#header 추가
header = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
response = requests.get(url, headers = header)
print(response.status_code) #406 -> 200

soup = bs(response.text, 'html.parser')
# print(soup)

#rank 추출 find_all 1
# span = soup.find_all('span', class_ ="rank").text
# print(span)

200
1


In [ ]:
#1위~50위 순위 리스트에 담아서 csv파일로 추출하기

li = []

songs = soup.find_all('tr', class_='lst50')
# print(songs)
# 모든 tr태그중 class= lst50에 있는 내용을 찾아 songs에 대입

for i in songs:
    ranks = (i.find_all('span', class_='rank')[0].text) + (i.find_all('span', class_='none')[0].text)
    #tr 태그의 class = lst50에 있는 모든 span 태그중 class rank인 0번째 위치한 것을
    #텍스트만 뽑고, span 태그중 class none인 0번째 위치한 것을 연결해서 ranks 대입
    # print(ranks)

    titles = i.find_all('a')[2].text
    #모든 a태그 중 2번째 위치한 것을 텍스트만 뽑아서 titles에 대입
    # print(titles)

    singers = i.find_all('a')[3].text
    #모든 a태그 중 3번째 위치한 것을 텍스트만 뽑아서 singers에 대입

    # print(singers)
    li.append([ranks, titles, singers])
    #빈 리스트에 순위, 노래제목, 가수명으로 추가
    # print(f"{ranks} 노래제목 : {titles} 가수명 : {singers}")

#open() ~ .close 함수 이용 csv파일로 만들기
f = open("melon.csv", "w", newline = '', encoding = 'utf-8')

writer = csv.writer(f)

col_names = [['순위', '노래제목', '가수명']]
for row in col_names:
    writer.writerow(row)

for row in li:
    writer.writerow(row)

f.close()
print("melon.csv 파일이 생성되었습니다")
# print(li)

melon.csv 파일이 생성되었습니다


In [ ]:
#melon.csv 파일 데이터 프레임으로 출력

import pandas as pd

df = pd.read_csv("melon.csv")

df
#0. 가수명이 방탄소년단인 행들 추출
# df = df[df["가수명"] == "방탄소년단"]
# df





,순위,노래제목,가수명
0,1위,Super Shy,NewJeans
1,2위,Seven (feat. Latto) - Clean Ver.,정국
2,3위,ETA,NewJeans
3,4위,퀸카 (Queencard),(여자)아이들
4,5위,I AM,IVE (아이브)
5,6위,헤어지자 말해요,박재정
6,7위,"이브, 프시케 그리고 푸른 수염의 아내",LE SSERAFIM (르세라핌)
7,8위,Spicy,aespa
8,9위,Steal The Show (From “엘리멘탈”),Lauv
9,10위,New Jeans,NewJeans


In [ ]:
#1. 가수별 노래 개수 출력
# singer_count = df['가수명'].value_counts()
# print(singer_count)

#2. 뉴진스 노래 개수 출력
# newjeans = df[df['가수명'] == "NewJeans"].value_counts()
# print(newjeans)
# print(len(newjeans))
# print(sum(newjeans))

#3. 가수별 최고 순위 찾기(임영웅, 아이브)
# singer_name1 = "임영웅"
# singer_name2 = "IVE (아이브)"

# df1 = df[df['가수명'] == singer_name1]
df2 = df[df['가수명'] == "IVE (아이브)"]

# max_rank1 = df1['순위'].min()
max_rank2 = df2['순위'].min()

# print(f"{singer_name1}의 최고순위 : {max_rank1}")
print(f"IVE (아이브)의 최고순위 : {max_rank2}")



IVE (아이브)의 최고순위 : 13위


In [ ]:
#4. 임영웅, 뉴진스, 아이브, (여자)아이들, 방탄소년단의 가수별 평균 순위
df = pd.read_csv("melon.csv")
df
df['순위'] = df['순위'].str.replace('위', '').astype(int)
df

#가수별 평균 순위 계산
singer_list = ['임영웅', 'NewJeans', '(여자)아이들', '방탄소년단']
singer_rank = df[df['가수명'].isin(singer_list)].groupby('가수명')['순위'].mean()

print(singer_rank)

가수명
(여자)아이들      4.000000
NewJeans    12.125000
방탄소년단       41.500000
임영웅         29.214286
Name: 순위, dtype: float64
